**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов

In [1]:
import re
import gzip

file_name = '../data/dostoevsky-besy-p2.txt.gz'

# загружаем текст ...
with gzip.open(file_name,'rt') as f:  
    text = f.read()[105:] # ...и выкидываем заголовок

display(f'символов:{len(text)}')
display(text[:364].strip())

'символов:465490'

'Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.'

In [2]:
from nltk import __version__ as nltk_version
display(f'nltk version: {nltk_version}')

'nltk version: 3.8.1'

In [3]:
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in nltk_sentence_split(text) # режем текст на отдельные предложения
]

display(f'предложений: {len(text)}')
display( sample(text,2) )

'предложений: 5556'

[['-', 'О', ',', 'какое', 'извержение', 'чужих', 'слов', '!'],
 ['Вы', 'мой', 'идол', '!']]

---

In [4]:
from itertools import chain
from collections import Counter

text_tokens = list(chain(*text)) # собираем все токены (слова) из текста
vocab_size = len(set(text_tokens)) # размер словаря

text_tokens_freq = Counter( text_tokens ) # оценка частоты использования слов

display(f'всего слов тексте: {len(text_tokens)}')
display(f'размер словаря: {vocab_size}') 
display( text_tokens_freq.most_common()[:10]) # наиболее частые токены

'всего слов тексте: 92240'

'размер словаря: 16631'

[(',', 9513),
 ('.', 4067),
 ('-', 3005),
 ('и', 2870),
 ('не', 1688),
 ('в', 1648),
 ('что', 1262),
 ('?', 893),
 ('на', 842),
 ('с', 838)]

----

In [5]:
# from nltk.util import bigrams
from nltk.util import ngrams as nltk_ngrams


# вынимаем все n-gram из текста
ngram_len = 2 # работаем с биграммами
text_ngrams = [ ngram for s in text for ngram in nltk_ngrams(s,ngram_len) ]
display( f'количество n-gram: {len(set(text_ngrams))}')
display( sample(text_ngrams,5) )

'количество n-gram: 54227'

[('что-то', 'уж'),
 ('болтали', '!'),
 ('быть', 'посетит'),
 ('угрюмым', 'видом'),
 (',', 'что')]

In [6]:
# cчитаем частоту n-gram
text_ngrams_freq = Counter(text_ngrams)
display( sample( list(text_ngrams_freq.items()), 7) )

[(('сама', 'поминутно'), 1),
 (('правительстве', '...'), 1),
 (('вы', 'поймете'), 1),
 (('Постойте', ','), 1),
 (('-', 'заметила'), 1),
 (('которых', 'еще'), 1),
 (('анекдота', ','), 1)]

In [7]:
display( text_ngrams_freq.most_common()[:10] ) # наиболее частые n-ngram

[((',', 'что'), 909),
 ((',', '-'), 699),
 ((',', 'и'), 495),
 ((',', 'а'), 431),
 ((',', 'но'), 351),
 ((',', 'как'), 235),
 ((',', 'я'), 209),
 ((',', 'не'), 206),
 ((',', 'в'), 189),
 (('-', 'Я'), 172)]

----

Оценка вероятностей совместного использования слов со сглаживанием Лапласа


$$
P(w_n|w_{n-1}) = \frac{ C(w_{n-1} w_{n}) +1 }{ C(w_{n-1}) + V }
$$

In [8]:
text_ngrams_prob = { # оценка совместного использования токенов
    ngram : (text_ngrams_freq[ngram]+1) / ( text_tokens_freq[ ngram[0] ] + vocab_size )
    for ngram in text_ngrams_freq 
}

display( sample( list(text_ngrams_prob.items()), 12) )

[((',', 'Евангелие'), 7.649938800489595e-05),
 (('служение', 'честной'), 0.00012025012025012025),
 ((',', 'во'), 0.0006119951040391676),
 ((',', 'поддерживаемая'), 7.649938800489595e-05),
 (('тоже', 'больше'), 0.00011953858107704261),
 (('и', 'умер'), 0.00010255884313624942),
 (('мелочь', 'и'), 0.00012025012025012025),
 (('``', 'франтящею'), 0.00011821728336682824),
 (('огонь', 'пойдет'), 0.00012022843402464683),
 (('лишь', 'только'), 0.0001798668984951136),
 (('до', 'жадности'), 0.00011912561796414319),
 (('и', 'глухой'), 0.00015383826470437414)]

In [9]:
# оценка вероятности новых ngram по статистике text_ngrams_prob
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

выбираем дополнение предложения с наибольшей вероятностью цепочки слов

$$
P(w_1,\ldots, w_n) = \prod_{k=1}^{n} P(w_k|w_k-1)
$$

In [10]:
from operator import mul
from functools import reduce 

# оценка предложения
def get_sentence_prob(
        sentence,
        text_ngrams_prob=text_ngrams_prob,
        text_tokens_freq=text_tokens_freq,
        ngram_len=ngram_len
    ):
    return reduce( mul, [ 
        get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in nltk_ngrams(sentence,ngram_len) 
    ] )


In [11]:
# оценка всех возможных продолжений предложения
def get_next_token_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    
    sentence_prob = get_sentence_prob( # оценка предложения
            sentence,
            text_ngrams_prob=text_ngrams_prob,
            text_tokens_freq=text_tokens_freq
        )
    
    token_next = { # оценки всех возможных продолжений
        ngram[1] : 
            sentence_prob*
              get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in text_ngrams_prob if ngram[0]==sentence[-1] 
    }
    
    return token_next


In [12]:
from collections import OrderedDict

def get_top_prob_token(tokens_prob,n=3):  # n наиболее вероятных продолжений
    kf = lambda k: tokens_prob[k]
    return OrderedDict([
        (key,tokens_prob[key])
        for key in sorted(tokens_prob, key=kf, reverse=True)[:n]
    ])

----

In [13]:
from IPython.display import HTML

def cstr(s, color='black'):
    return f'<font style="color:{color};">{s}</font>'

str_init_len = 9
words_predict = 5

# генерируем продолжения
for sentence in sample(text,20): # выбираем рандомно N предложений
    if len(sentence)<str_init_len+words_predict: continue # выкидываем очень короткие предложения
    sentence_ = sentence[:str_init_len] # берём начало предложения
    
    # считаем верояности продолжений
    next_token_prob = get_next_token_prob(sentence_) 
    
    # выбираем наиболее вероятные
    top_prob_token = get_top_prob_token(next_token_prob,n=words_predict)
    result = ' '.join(top_prob_token.keys())
      
    display( HTML( cstr( ' '.join(sentence_)) + ' -> ' + cstr( result, color='red') ) ) 